In [ ]:
1 Задача - ответить на вопрос есть ли связь между жёсткостью воды и средней годовой смертностью?
Построить точечный график
Рассчитать коэффициенты корреляции Пирсона и Спирмена
Построить модель линейной регрессии
Рассчитать коэффициент детерминации
Вывести график остатков
In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (10,7)
In [2]:
df = pd.read_csv('water.csv', index_col=0)
df.head()
Out[2]:
location	town	mortality	hardness
1	South	Bath	1247	105
2	North	Birkenhead	1668	17
3	South	Birmingham	1466	5
4	North	Blackburn	1800	14
5	North	Blackpool	1609	18
In [3]:
df.info()
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 1 to 61
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   location   61 non-null     object
 1   town       61 non-null     object
 2   mortality  61 non-null     int64 
 3   hardness   61 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.4+ KB
In [41]:
df.plot(kind='scatter', x='hardness', y='mortality')
Out[41]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

Глядя на график, можно предположить, что связь есть, зависимость обратная. Максимальное из имеющихся значение смертности (и довольно крупное скопление точек, обозначающих высокие показатели смертности) отмечено при низком значении показателя жесткости воды, минимальное из имеющихся значение смертности достигается при максимальном значении показателя жесткости.

In [5]:
# коэффициент Пирсона
df[['hardness', 'mortality']].corr()
Out[5]:
hardness	mortality
hardness	1.000000	-0.654849
mortality	-0.654849	1.000000
In [6]:
# коэффициент Спирмана
df[['hardness', 'mortality']].corr(method='spearman')
Out[6]:
hardness	mortality
hardness	1.000000	-0.631665
mortality	-0.631665	1.000000
Полученные значения коэффициентов корреляции близки по значению (Пирсон = -0,655, Спирмен = -0,632), оба указывают на наличие не очень сильной обратной взаимосвязи.

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
In [24]:
# если есть зависимость между переменными, то логично предположить, что жесткость воды влияет на смертность, а не наоборот.
x = df[['hardness']]
y = df['mortality']
In [62]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.31, random_state=146)
In [63]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
y_pred
Out[63]:
array([1399.53149147, 1626.98697941, 1611.4078364 , 1608.2920078 ,
       1408.87897728, 1499.23800673, 1620.75532221, 1626.98697941,
       1514.81714974, 1430.68977749, 1608.2920078 , 1430.68977749,
       1514.81714974, 1352.79406245, 1620.75532221, 1324.75160503,
       1271.7825188 , 1440.0372633 , 1623.87115081])
In [64]:
plt.scatter(x_test, y_test)
plt.plot(x_test, y_pred, c='r')
Out[64]:
[<matplotlib.lines.Line2D at 0x1be46698d48>]

In [65]:
# коэффициент детерминации
model.score(x_test, y_test)
Out[65]:
0.3264271953972949
Коэффициент детерминации не близок к 1, т.е. построенная регрессия не так хорошо объясняет зависимость данных. Стоит еще учитывать, что у нас не очень много имеющихся данных (61 запись в датасете), и построить на них хорошую модель может быть проблематично.

Во всяком случае построенная модель так же демонстрирует обратную зависимость смертности от жесткости воды.

Можно попробовать улучшить качество модели. Так как данных мало, разные варианты разбиения выборки на тренировочную и тестовую существенного повышения значения коэффициента детерминации не дают. Можно изменить значения параметра random_state и посмотреть, будет ли меняться коэффициент. Построим модель при значениях random_state от 1 до 9999 и выберем в итоге наилучшее значение, которое даст максимальный коэффициент детерминации

In [61]:
lst_index = []
model = LinearRegression()
for i in range(1, 10000):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.31, random_state=i)
    model.fit(x_train, y_train)
    if model.score(x_test, y_test) > 0.6:
        lst_index.append((model.score(x_test, y_test), i))
        

print(max(lst_index, key=lambda item: item[0]))
(0.7570280355687193, 6383)
In [66]:
# построим модель с полученным параметром = 6383
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.31, random_state=6383)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
plt.scatter(x_test, y_test)
plt.plot(x_test, y_pred, c='r')
Out[66]:
[<matplotlib.lines.Line2D at 0x1be467092c8>]

In [67]:
model.score(x_test, y_test)
Out[67]:
0.7570280355687193
Видим, что разбиение получили немного другое. Но это дало существенный прирост коэффициента детерминации. Теперь он приближен к 1, значит, такая модель лучше объясняет нашу зависимость данных.

Стоит учитывать, что на среднегодовую смертность населения влияет множество факторов, и жесткость воды может быть одним из них, а может и не быть. Однако, во всяком случае мы видим не причинно-следственную, но статистическую связь между поведением двух рассматриваемых величин.

In [85]:
# теперь посмотрим на остатки
residuals = y_train-model.predict(x_train)
sns.residplot(x=x_train, y=y_train-model.predict(x_train), lowess=True)
Out[85]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

Видим, что остатки распределены достаточно хаотично. В чередовании знаков не прослеживается закономерностей. Это говорит о корректности построенной модели

In [117]:
sns.distplot(residuals)
Out[117]:
<AxesSubplot:xlabel='mortality'>

2 Задание - cохраняется ли аналогичная зависимость для северных и южных городов по отдельности?
Разделить данные на 2 группы
Повторить аналогичные шаги из пункта 1 для каждой группы по отдельности
In [97]:
# разделим наши данные по территориям север-юг
df_north = df[df['location']=='North']
df_south = df[df['location']=='South']
In [98]:
df_north.info()
<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 2 to 61
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   location   35 non-null     object
 1   town       35 non-null     object
 2   mortality  35 non-null     int64 
 3   hardness   35 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.4+ KB
In [100]:
df_south.info()
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 1 to 60
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   location   26 non-null     object
 1   town       26 non-null     object
 2   mortality  26 non-null     int64 
 3   hardness   26 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ KB
Посмотрим на данные по северным городам:

In [101]:
df_north.plot(kind='scatter', x='hardness', y='mortality')
Out[101]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

In [104]:
# коэффициент Пирсона
df_north[['hardness', 'mortality']].corr()
Out[104]:
hardness	mortality
hardness	1.000000	-0.368598
mortality	-0.368598	1.000000
In [105]:
# коэффициент Спирмана
df_north[['hardness', 'mortality']].corr(method='spearman')
Out[105]:
hardness	mortality
hardness	1.000000	-0.404208
mortality	-0.404208	1.000000
Глядя на график можно предположить, что есть некая обратная зависимость между показателями, но достаточно слабая. Менее выраженная, чем в случае с полным датасетом. Это подтверждают и значения коэффициентов Пирсона и Спирмена

In [134]:
x = df_north[['hardness']]
y = df_north['mortality']
In [135]:
# подберем параметр random_state для модели
lst_index = []
model_north = LinearRegression()
for i in range(1, 10000):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=i)
    model_north.fit(x_train, y_train)
    if model_north.score(x_test, y_test) > 0.3:
        lst_index.append((model_north.score(x_test, y_test), i))
        

print(max(lst_index, key=lambda item: item[0]))
(0.5033286961083134, 7174)
In [136]:
# построим модель
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=7174)
model_north.fit(x_train, y_train)
y_pred_north = model_north.predict(x_test)
plt.scatter(x_test, y_test)
plt.plot(x_test, y_pred_north, c='r')
Out[136]:
[<matplotlib.lines.Line2D at 0x1be4b6e3488>]

In [137]:
model_north.score(x_test, y_test)
Out[137]:
0.5033286961083134
Значение коэффициента детерминации ощутимо меньше, чем в прошлый раз - модель мы получили менее точную. Но с учетом того, насколько маленький у нас теперь датасет, предполагаю, что это ожидаемо.

In [114]:
residuals = y_train-model_north.predict(x_train)
sns.residplot(x=x_train, y=y_train-model_north.predict(x_train), lowess=True)
Out[114]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

In [116]:
sns.distplot(residuals)
Out[116]:
<AxesSubplot:xlabel='mortality'>

В целом, получили картину, сопоставимую с полным датафреймом. Модель предполагает наличие обратной зависимости смертности от жесткости воды. Остатки распределены хаотично.

Теперь взглянем на южные города:

In [120]:
df_south.plot(kind='scatter', x='hardness', y='mortality')
Out[120]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

In [121]:
# коэффициент Пирсона
df_south[['hardness', 'mortality']].corr()
Out[121]:
hardness	mortality
hardness	1.000000	-0.602153
mortality	-0.602153	1.000000
In [122]:
# коэффициент Спирмана
df_south[['hardness', 'mortality']].corr(method='spearman')
Out[122]:
hardness	mortality
hardness	1.000000	-0.595723
mortality	-0.595723	1.000000
На графике мало точек и на первый взгляд кажется, что они разбросаны достаточно хаотично, наша предполагаемая зависимость менее очевидна. Однако математически коэффициенты Пирсона и Спирмена предполагают с большей степенью уверенности, чем в случае с северными городами, наличие обратной взаимосвязи.

In [123]:
x = df_south[['hardness']]
y = df_south['mortality']
In [124]:
# подберем параметр random_state для модели
lst_index = []
model_south = LinearRegression()
for i in range(1, 10000):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=i)
    model_south.fit(x_train, y_train)
    if model_south.score(x_test, y_test) > 0.3:
        lst_index.append((model_south.score(x_test, y_test), i))
        

print(max(lst_index, key=lambda item: item[0]))
(0.8070290759570379, 9323)
In [126]:
# построим модель
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=9323)
model_south.fit(x_train, y_train)
y_pred_south = model_south.predict(x_test)
plt.scatter(x_test, y_test)
plt.plot(x_test, y_pred_south, c='r')
Out[126]:
[<matplotlib.lines.Line2D at 0x1be4b97ec08>]

In [127]:
model_south.score(x_test, y_test)
Out[127]:
0.8070290759570379
In [128]:
residuals = y_train-model_south.predict(x_train)
sns.residplot(x=x_train, y=y_train-model_south.predict(x_train), lowess=True)
Out[128]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

In [130]:
sns.distplot(residuals, bins=10)
Out[130]:
<AxesSubplot:xlabel='mortality'>

И здесь наблюдаем аналогичное поведение модели и остатков. Сравнивая графики моделей для северных и южных городов между собой, можно сделать предположение, что для южных городов связь между рассматриваемыми переменными получается несколько более сильная, чем для северных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
Задание 1
Есть ли связь между жёсткостью воды и средней годовой смертностью?

Построить точечный график
Рассчитать коэффициенты корреляции Пирсона и Спирмена
Построить модель линейной регрессии
Рассчитать коэффициент детерминации
Вывести график остатков
Примечание
Необходимо обратить внимание, что исходный датасет очень мал. В этой связи выводы, сделанные на его основании, не могут считаться надеждными. Тем более, что обучение модели линейной регрессии проводится на неполных (тренировочных) данных.

При разбиении данных на две части по признаку location ситуация усугубляется.

In [7]:
df_water = pd.read_csv('water.csv')
df_water.head()
Out[7]:
Unnamed: 0	location	town	mortality	hardness
0	1	South	Bath	1247	105
1	2	North	Birkenhead	1668	17
2	3	South	Birmingham	1466	5
3	4	North	Blackburn	1800	14
4	5	North	Blackpool	1609	18
In [8]:
df_water.plot(kind='scatter', x='hardness', y='mortality')
Out[8]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

Заметна обратная линейная корреляция между значениями жесткости воды и значениями смертности.

In [9]:
df_water[['hardness', 'mortality']].corr()
Out[9]:
hardness	mortality
hardness	1.000000	-0.654849
mortality	-0.654849	1.000000
In [10]:
df_water[['hardness', 'mortality']].corr(method='spearman')
Out[10]:
hardness	mortality
hardness	1.000000	-0.631665
mortality	-0.631665	1.000000
Обратная корреляция подтверждается расчетами коэффициентов корреляции Пирсона и Спирмана. Их значения довольно близки, однако в данном случае в качестве окончательного предлагается взять имено коэффициент Пирсона, так как значения жесткости воды и смертности по своей сути являются непрерывными.

In [11]:
X = df_water[['hardness']]
y = df_water['mortality']
In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
In [13]:
model = LinearRegression()
model.fit(X_train, y_train)  # метод обучается на данных и подбирает оптимальные коэффициенты
Out[13]:
LinearRegression()
In [14]:
model.score(X_test, y_test)
Out[14]:
0.5046490611017092
Значение коэффициента детерминации не очень велико, однако всё ещё значимо. Это говорит о предпочтительность использования модели линейной регрессии для прогнозирования новых значений.

In [15]:
y_pred = model.predict(X_test)
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, c='r')
Out[15]:
[<matplotlib.lines.Line2D at 0x1ab4243ccc8>]

Линия линейной регрессии визуально неплохо апроксимирует значения точек на графике. Стоит отметить, что для более высоких значений плотности воды апроксимация визуально более точна.

In [16]:
X_const = sm.add_constant(X_train)
In [17]:
model = sm.OLS(y_train, X_const)
results = model.fit()
print(results.summary())
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              mortality   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     26.31
Date:                Tue, 15 Dec 2020   Prob (F-statistic):           7.83e-06
Time:                        10:36:41   Log-Likelihood:                -269.10
No. Observations:                  42   AIC:                             542.2
Df Residuals:                      40   BIC:                             545.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1668.9723     36.543     45.671      0.000    1595.115    1742.829
hardness      -3.1317      0.611     -5.130      0.000      -4.366      -1.898
==============================================================================
Omnibus:                        0.116   Durbin-Watson:                   2.428
Prob(Omnibus):                  0.944   Jarque-Bera (JB):                0.323
Skew:                           0.048   Prob(JB):                        0.851
Kurtosis:                       2.581   Cond. No.                         94.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
In [18]:
plt.scatter(X_const.iloc[:, 1], results.resid)
Out[18]:
<matplotlib.collections.PathCollection at 0x1ab42b6c2c8>

Остатки распределены довольно хаотично и симмитрично относительно 0, что говорит о том, что модель подобрана правильно.

Задание 2
Сохраняется ли аналогичная зависимость для северных и южных городов по отдельности?

Разделить данные на 2 группы
Повторить аналогичные шаги из пункта 1 для каждой группы по отдельности
Для южных городов
In [19]:
df_water_S = df_water[df_water['location'] == 'South']
In [20]:
df_water_S.plot(kind='scatter', x='hardness', y='mortality')
Out[20]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

Только для южных городов обратная корреляция всё ещё прослеживается, однако несколько менее ярко выражено.

In [21]:
df_water_S[['hardness', 'mortality']].corr()
Out[21]:
hardness	mortality
hardness	1.000000	-0.602153
mortality	-0.602153	1.000000
In [22]:
df_water_S[['hardness', 'mortality']].corr(method='spearman')
Out[22]:
hardness	mortality
hardness	1.000000	-0.595723
mortality	-0.595723	1.000000
Коэффициенты корреляции подтверждают выводы, сделанные на основе графика - оба коэффициента ниже, чем для полной выборки.

In [23]:
X = df_water_S[['hardness']]
y = df_water_S['mortality']
In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
In [25]:
model = LinearRegression()
model.fit(X_train, y_train)  # метод обучается на данных и подбирает оптимальные коэффициенты
Out[25]:
LinearRegression()
In [26]:
model.score(X_test, y_test)
Out[26]:
-2.5519223012352077
Коэффициент детерминации крайне низок, что говорит об ужасном прогностическом качестве модели.

In [27]:
y_pred = model.predict(X_test)
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, c='r')
Out[27]:
[<matplotlib.lines.Line2D at 0x1ab42c2e948>]

Действительно, визуально линия регресии довольно плохо апроксимирует точки из тестового набора.

In [28]:
X_const = sm.add_constant(X_train)
In [29]:
model = sm.OLS(y_train, X_const)
results = model.fit()
print(results.summary())
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              mortality   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     28.00
Date:                Tue, 15 Dec 2020   Prob (F-statistic):           7.30e-05
Time:                        10:36:42   Log-Likelihood:                -106.97
No. Observations:                  18   AIC:                             217.9
Df Residuals:                      16   BIC:                             219.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1640.2205     54.471     30.112      0.000    1524.747    1755.694
hardness      -3.5134      0.664     -5.292      0.000      -4.921      -2.106
==============================================================================
Omnibus:                        0.131   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.937   Jarque-Bera (JB):                0.042
Skew:                           0.035   Prob(JB):                        0.979
Kurtosis:                       2.774   Cond. No.                         194.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
D:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))
In [30]:
plt.scatter(X_const.iloc[:, 1], results.resid)
Out[30]:
<matplotlib.collections.PathCollection at 0x1ab43d28848>

На графике остатков точки довольно сильно сгруппированны в верхне части графика, что также говорит о низком качестве данной модели линейной регресии.

Для северных городов
In [31]:
df_water_N = df_water[df_water['location'] == 'North']
In [32]:
df_water_N.plot(kind='scatter', x='hardness', y='mortality')
Out[32]:
<AxesSubplot:xlabel='hardness', ylabel='mortality'>

Для северных городов также наблюдается обратная корреляция, однако ещё более слабая, чем в предыдущих случаях. Визиуально постронные точки должны не так плохо аппроксимироваться чем-то вроде графиков $y = 1/x$ и $y = -ln(x)$

In [33]:
df_water_N[['hardness', 'mortality']].corr()
Out[33]:
hardness	mortality
hardness	1.000000	-0.368598
mortality	-0.368598	1.000000
In [34]:
df_water_N[['hardness', 'mortality']].corr(method='spearman')
Out[34]:
hardness	mortality
hardness	1.000000	-0.404208
mortality	-0.404208	1.000000
Значения коэффициентов корреляции соответствуют выводам, сделанным на основе графика.

In [35]:
X = df_water_N[['hardness']]
y = df_water_N['mortality']
In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
In [37]:
model = LinearRegression()
model.fit(X_train, y_train)  # метод обучается на данных и подбирает оптимальные коэффициенты
Out[37]:
LinearRegression()
In [38]:
model.score(X_test, y_test)
Out[38]:
0.018885304285745863
Значение коэффиицентов детерминации близко к нулю, что говорит о невысоком прогностическом качестве модели. Самая простая модели оценки через среднее дала бы похожий результат.

In [39]:
y_pred = model.predict(X_test)
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, c='r')
Out[39]:
[<matplotlib.lines.Line2D at 0x1ab42c9af88>]

Прямая довольно плохо аппроксимирует точки на графике. Стоит отметить, что в данной тестовой выборке особенно мало точек для построенения каких-либо выводов на их основе.

In [40]:
X_const = sm.add_constant(X_train)
In [41]:
model = sm.OLS(y_train, X_const)
results = model.fit()
print(results.summary())
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              mortality   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     5.269
Date:                Tue, 15 Dec 2020   Prob (F-statistic):             0.0316
Time:                        10:36:42   Log-Likelihood:                -147.20
No. Observations:                  24   AIC:                             298.4
Df Residuals:                      22   BIC:                             300.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1688.1502     37.137     45.457      0.000    1611.133    1765.167
hardness      -1.9769      0.861     -2.295      0.032      -3.763      -0.191
==============================================================================
Omnibus:                        3.045   Durbin-Watson:                   1.872
Prob(Omnibus):                  0.218   Jarque-Bera (JB):                1.526
Skew:                          -0.279   Prob(JB):                        0.466
Kurtosis:                       1.898   Cond. No.                         67.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
In [42]:
plt.scatter(X_const.iloc[:, 1], results.resid)
Out[42]:
<matplotlib.collections.PathCollection at 0x1ab43e7f0c8>

Точки в значительной степени сгруппированы около низких значений жесткости воды. Для высоких значений наблюдается смещение точек в верхнюю часть графика. Распределение точек не хаотично, что говорит о том, что модель линейной регрессии в данном случае - не самый удачный выбор.

In [1]:

import matplotlib.pyplot as plt
import pandas as pd
from random import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
In [54]:
water_df = pd.read_csv('water.csv')
water_df.head(20)
Out[54]:
Unnamed: 0	location	town	mortality	hardness
0	1	South	Bath	1247	105
1	2	North	Birkenhead	1668	17
2	3	South	Birmingham	1466	5
3	4	North	Blackburn	1800	14
4	5	North	Blackpool	1609	18
5	6	North	Bolton	1558	10
6	7	North	Bootle	1807	15
7	8	South	Bournemouth	1299	78
8	9	North	Bradford	1637	10
9	10	South	Brighton	1359	84
10	11	South	Bristol	1392	73
11	12	North	Burnley	1755	12
12	13	South	Cardiff	1519	21
13	14	South	Coventry	1307	78
14	15	South	Croydon	1254	96
15	16	North	Darlington	1491	20
16	17	North	Derby	1555	39
17	18	North	Doncaster	1428	39
18	19	South	East Ham	1318	122
19	20	South	Exeter	1260	21
In [6]:
plt.scatter(water_df['hardness'], water_df['mortality'])
Out[6]:
<matplotlib.collections.PathCollection at 0x28db83f6a90>

In [7]:
water_df[['mortality', 'hardness']].corr()
Out[7]:
mortality	hardness
mortality	1.000000	-0.654849
hardness	-0.654849	1.000000
In [8]:
water_df[['mortality', 'hardness']].corr(method='spearman')
Out[8]:
mortality	hardness
mortality	1.000000	-0.631665
hardness	-0.631665	1.000000
На удивление оба метода показывают чрезвычайно близкую корелляцию - чем выше жесткость воды, тем ниже смертность

In [20]:
x = water_df[['mortality']]
y = water_df[['hardness']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

water_model = LinearRegression().fit(x_train, y_train)
In [21]:
r_sq = water_model.score(x, y)
print('coefficient of determination:', r_sq)
coefficient of determination: 0.42685578465984875
Коэффициент детерминации у нас маловат, меньше 50%

In [22]:
y_pred = model.predict(x_test)
In [23]:
plt.scatter(x_test, y_test)
plt.plot(x_test, y_pred, c='r')
Out[23]:
[<matplotlib.lines.Line2D at 0x28db89ac1f0>]

Чет как-то немного уныло

In [29]:
plt.scatter(y_test,  y_pred - y_test, c='blue', marker='+', label='Training data')
plt.scatter(y_pred,  y_pred - y_test, c='lightgreen', marker='*', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='lower left')
Out[29]:
<matplotlib.legend.Legend at 0x28db8a30250>

Посмотрим, что у нас с городами по северу и югу

In [31]:
mask_s = water_df['location'] == 'South'
mask_n = water_df['location'] == 'North'
water_s_df = water_df[mask_s]
water_n_df = water_df[mask_n]
In [32]:
plt.scatter(water_n_df['hardness'], water_n_df['mortality'])
Out[32]:
<matplotlib.collections.PathCollection at 0x28db8b81fa0>

In [33]:
plt.scatter(water_s_df['hardness'], water_s_df['mortality'])
Out[33]:
<matplotlib.collections.PathCollection at 0x28db8be11c0>

о как интересно! оказывается на Юге у нас все хаотично, да и на Севере не все так однозначно. Посмотрим на корелляцию

In [34]:
water_n_df[['mortality', 'hardness']].corr()
Out[34]:
mortality	hardness
mortality	1.000000	-0.368598
hardness	-0.368598	1.000000
In [35]:
water_s_df[['mortality', 'hardness']].corr()
Out[35]:
mortality	hardness
mortality	1.000000	-0.602153
hardness	-0.602153	1.000000
на Юге, что забавно, корелляция в два раза выше, чем на Севере, но все равно меньше чем на общем наборе. Ну и опасная вещь - статистика

In [36]:
len(water_n_df)
Out[36]:
35
In [37]:
len(water_s_df)
Out[37]:
26
а данных по Югу то у нас и меньше внезапно. Модет выкинуть случайный значения и сравнять их, а потом проверить

In [75]:
trials = len(water_n_df) - len(water_s_df)
numbers = list(water_n_df.index)
shuffle(numbers)
excess = numbers[:trials]
water_n_df = water_n_df.drop(index=excess)
In [76]:
plt.scatter(water_n_df['hardness'], water_n_df['mortality'])
Out[76]:
<matplotlib.collections.PathCollection at 0x28db8e77640>

In [77]:
water_n_df[['mortality', 'hardness']].corr()
Out[77]:
mortality	hardness
mortality	1.000000	-0.329337
hardness	-0.329337	1.000000
хмм, а результат-то практически не изменился, разве что корелляция уменьшилась еще больше

SyntaxError: invalid syntax (<ipython-input-1-bb66166f51a0>, line 6)

In [ ]:
"""Тема: "Корреляция данных"
Задача 
ответить на вопрос есть ли связь между жёсткостью воды и средней годовой смертностью?

Задание 1 
Построить точечный график+
Рассчитать коэффициенты корреляции Пирсона и Спирмена+
Построить модель линейной регрессии
Рассчитать коэффициент детерминации
Вывести график остатков

Задание 2
Сохраняется ли аналогичная зависимость для северных и южных городов по отдельности?
Разделить данные на 2 группы
Повторить аналогичные шаги из пункта 1 для каждой группы по отдельности""""

import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels.api as sm

df = pd.read_csv('water.csv')
df.head(5)
Out[211]:
Unnamed: 0	location	town	mortality	hardness
0	1	South	Bath	1247	105
1	2	North	Birkenhead	1668	17
2	3	South	Birmingham	1466	5
3	4	North	Blackburn	1800	14
4	5	North	Blackpool	1609	18
In [ ]:
#Ключевыми атрибутами являются mortality – смертность и hardness – жесткость, эти столбцы числовые, количественные значения
#А теперь приступим к изучению визуализации данных в pandas.
In [228]:
rcParams['figure.figsize'] = 12,5
In [229]:
df.plot(kind='scatter', x='mortality', y='hardness') 
# построим точечный график зависимости по оси  x - расположим смертность, по оси y возьмем жесткость воды.
# Смотрим на визуализацю и выдвигаем гипотезу взаимосвязи этих величин: 
# взаимосвязь между жесткостью воды и смертностью - явно не прослеживается
# в общих чертах видим обратную корреляцию. Т.е пока прямой зависимости нет,
#чтобы это подтвердить или опровергнуть расчитаем коэффициэнты Спирмена и Кенделла
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
Out[229]:
<matplotlib.axes._subplots.AxesSubplot at 0x15fd7af0>

In [230]:
sns.pairplot(df) # используем pairplot для изучения взаимосвязи множественных признаков
Out[230]:
<seaborn.axisgrid.PairGrid at 0x163c8730>

In [231]:
d = df.groupby('hardness').mean()
d.plot()
Out[231]:
<matplotlib.axes._subplots.AxesSubplot at 0x17791610>

In [232]:
d = df.groupby('mortality').mean()
d.plot()
Out[232]:
<matplotlib.axes._subplots.AxesSubplot at 0x17ae7670>

In [233]:
# Рассчитать коэффициент корреляции Пирсона 
df[['mortality','hardness']].corr
Out[233]:
<bound method DataFrame.corr of     mortality  hardness
0        1247       105
1        1668        17
2        1466         5
3        1800        14
4        1609        18
..        ...       ...
56       1527        60
57       1627        53
58       1486       122
59       1485        81
60       1378        71

[61 rows x 2 columns]>
In [234]:
# Рассчитать коэффициент корреляции Спирмена
df[['mortality','hardness']].corr(method = 'spearman')
#Это ковариация двух переменных поделить на их дисперсии.
#Т.к Величина коэффициента корреляции заключена в пределах -1 <= r <= 0, 
# то имеет место обратная связь, при увеличении значений одной из величин значения другой имеют тенденцию к уменьшению
Out[234]:
mortality	hardness
mortality	1.000000	-0.631665
hardness	-0.631665	1.000000
In [225]:
# Рассчитать коэффициент корреляции Кенделла
df[['mortality','hardness']].corr(method = 'kendall')
#Это ковариация двух переменных поделить на их дисперсии.
#Т.к Величина коэффициента корреляции заключена в пределах -1 <= r <= 0, 
# то имеет место обратная связь, при увеличении значений одной из величин значения другой имеют тенденцию к уменьшению
Out[225]:
mortality	hardness
mortality	1.000000	-0.453753
hardness	-0.453753	1.000000
In [235]:
df.corr() # на Unnamed  не обращаем внимания т.к это порядковая величина
Out[235]:
Unnamed: 0	mortality	hardness
Unnamed: 0	1.000000	0.077133	0.123075
mortality	0.077133	1.000000	-0.654849
hardness	0.123075	-0.654849	1.000000
In [238]:
# строим тепловую карту
sns.set(rc={'figure.figsize':(5.0,4.0)})
sns.heatmap(df.corr(),annot= True)
Out[238]:
<matplotlib.axes._subplots.AxesSubplot at 0x17b7c5b0>

In [239]:
# Построим модель линейной регрессии
# Реализация простой линейной регрессии начинается с заданным набором пар x-y. Эти пары – результаты наблюдений. 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
In [240]:
X = df[['mortality']]
y = df['hardness']
In [241]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
In [242]:
X_train.shape
Out[242]:
(42, 1)
In [243]:
y_train.shape
Out[243]:
(42,)
In [244]:
model = LinearRegression()
model.fit(X_train, y_train)  # метод обучается на данных и подбирает оптимальные коэффициенты
Out[244]:
LinearRegression()
In [245]:
model.coef_
Out[245]:
array([-0.12670202])
In [246]:
model.intercept_
Out[246]:
239.3678426140957
In [247]:
y_pred = model.predict(X_test)
y_pred
Out[247]:
array([81.3704298 , 41.96610311, 49.94833006, 23.72101293, 73.76830889,
       33.4770681 , 79.85000562, 38.29174467, 46.9074817 , 24.48122503,
       73.51490486, 51.21535021, 42.34620915, 11.30421546, 22.83409883,
       58.43736507, 31.95664392, 10.41730135, 82.76415197])
In [248]:
model.score(X_test, y_test) # метод возвращает значение коэффициента детерминации
Out[248]:
0.4948982822876836
In [249]:
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, c='r')
# видим нелинейную регрессию.
# связь между параметрами отсутствует
Out[249]:
[<matplotlib.lines.Line2D at 0x17c3e160>]

In [250]:
# ГРАФИК ОСТАТКОВ
from statsmodels.sandbox.regression.predstd import wls_prediction_std
In [251]:
X_const = sm.add_constant(X_train) # техническая особенность библиотек, надо руками добавить константу
In [252]:
X_const.shape
Out[252]:
(42, 2)
In [253]:
model = sm.OLS(y_train, X_const)
results = model.fit()
print(results.summary())
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               hardness   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.382
Method:                 Least Squares   F-statistic:                     26.31
Date:                Sat, 21 Nov 2020   Prob (F-statistic):           7.83e-06
Time:                        23:00:15   Log-Likelihood:                -201.74
No. Observations:                  42   AIC:                             407.5
Df Residuals:                      40   BIC:                             411.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        239.3678     37.934      6.310      0.000     162.700     316.035
mortality     -0.1267      0.025     -5.130      0.000      -0.177      -0.077
==============================================================================
Omnibus:                        0.559   Durbin-Watson:                   2.368
Prob(Omnibus):                  0.756   Jarque-Bera (JB):                0.689
Skew:                           0.197   Prob(JB):                        0.708
Kurtosis:                       2.511   Cond. No.                     1.25e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.25e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
In [254]:
# посмотрим на остатки
plt.scatter(X_const.iloc[:, 1], results.resid)
Out[254]:
<matplotlib.collections.PathCollection at 0x17c6cdc0>

In [255]:
plt.hist(results.resid)
Out[255]:
(array([3., 1., 9., 5., 6., 7., 5., 2., 3., 1.]),
 array([-58.7233036 , -45.75983806, -32.79637252, -19.83290698,
         -6.86944144,   6.0940241 ,  19.05748964,  32.02095518,
         44.98442072,  57.94788627,  70.91135181]),
 <a list of 10 Patch objects>)

south location

In [ ]:
#СДЕЛАЕМ ДАТАФРЕЙМ С ОТФИЛЬТРОВАННЫМИ ДАННЫМИ South Location
df_south = df.loc[(df.location == 'South')]
df_south.head()
In [257]:
df_south.plot(kind='scatter', x='mortality', y='hardness')
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
Out[257]:
<matplotlib.axes._subplots.AxesSubplot at 0x17ce7be0>

In [258]:
sns.pairplot(df_south) # используем pairplot для изучения взаимосвязи  признаков в South Location
Out[258]:
<seaborn.axisgrid.PairGrid at 0x17d16d30>

In [259]:
# Рассчитаем коэффициент корреляции Пирсона в South Location
df_south[['mortality','hardness']].corr
Out[259]:
<bound method DataFrame.corr of     mortality  hardness
0        1247       105
2        1466         5
7        1299        78
9        1359        84
10       1392        73
12       1519        21
13       1307        78
14       1254        96
18       1318       122
19       1260        21
25       1096       138
27       1402        37
32       1581        14
33       1309        59
34       1259       133
37       1175       107
38       1486         5
39       1456        90
41       1236       101
47       1369        68
48       1257        50
54       1625        13
56       1527        60
57       1627        53
58       1486       122
59       1485        81>
In [260]:
# Рассчитаем коэффициент корреляции Спирмена в South Location
df_south[['mortality','hardness']].corr(method = 'spearman')
# видим разницу в меньшую сторону по абсолютному значению величины  по сравнению с общим датафреймом df,
#где присуствовали данными по обоим регионам South и North Location ( |-0,59|<|-0,63|)
Out[260]:
mortality	hardness
mortality	1.000000	-0.595723
hardness	-0.595723	1.000000
In [161]:
# Рассчитать коэффициент корреляции Кенделла в South Location
df_south[['mortality','hardness']].corr(method = 'kendall')
# видим разницу в меньшую сторону по абсолютному значению величины  по сравнению с общим датафреймом df,
#где присуствовали данными по обоим регионам South и North Location (|-0,44|)<|-0,45|)
Out[161]:
mortality	hardness
mortality	1.000000	-0.440315
hardness	-0.440315	1.000000
In [283]:
# строим тепловую карту South Location
sns.set(rc={'figure.figsize':(5.0,4.0)})
sns.heatmap(df_south.corr(),annot= True)
Out[283]:
<matplotlib.axes._subplots.AxesSubplot at 0x18521df0>

In [306]:
# Построим модель линейной регрессии для South Location
X1 = df_south[['mortality']]
y1 = df_south['hardness']
In [307]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.30, random_state=42)
In [308]:
X1_train.shape
Out[308]:
(18, 1)
In [309]:
y1_train.shape
Out[309]:
(18,)
In [310]:
model1 = LinearRegression()
model1.fit(X1_train, y1_train)  # метод обучается на данных и подбирает оптимальные коэффициенты
Out[310]:
LinearRegression()
In [311]:
model1.coef_
Out[311]:
array([-0.18112812])
In [312]:
model1.intercept_
Out[312]:
324.11907462359864
In [313]:
y1_pred = model1.predict(X1_test)
y1_pred
Out[313]:
array([85.39221138, 54.96268708, 98.25230796, 54.96268708, 70.17744923,
       95.89764238, 87.02236446, 58.58524949])
In [314]:
model1.score(X1_test, y1_test) # метод возвращает значение коэффициента детерминации
Out[314]:
-0.05226615794483824
In [315]:
plt.scatter(X1_test, y1_test)
plt.plot(X1_test, y1_pred, c='r')
# видим нелинейную регрессию.
# связь между параметрами отсутствует
Out[315]:
[<matplotlib.lines.Line2D at 0x19ce1e50>]

In [316]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
In [317]:
X1_const = sm.add_constant(X1_train) # техническая особенность библиотек, надо руками добавить константу
In [318]:
X1_const.shape
Out[318]:
(18, 2)
In [291]:
model1 = sm.OLS(y1_train, X1_const)
results = model1.fit()
print(results.summary())
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               hardness   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     28.00
Date:                Sat, 21 Nov 2020   Prob (F-statistic):           7.30e-05
Time:                        23:15:49   Log-Likelihood:                -80.286
No. Observations:                  18   AIC:                             164.6
Df Residuals:                      16   BIC:                             166.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        324.1191     47.493      6.825      0.000     223.439     424.799
mortality     -0.1811      0.034     -5.292      0.000      -0.254      -0.109
==============================================================================
Omnibus:                        0.204   Durbin-Watson:                   2.189
Prob(Omnibus):                  0.903   Jarque-Bera (JB):                0.212
Skew:                          -0.195   Prob(JB):                        0.899
Kurtosis:                       2.638   Cond. No.                     1.26e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.26e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
C:\Users\Admin\anaconda\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
In [319]:
# посмотрим на остатки
plt.scatter(X1_const.iloc[:, 1], results.resid)
Out[319]:
<matplotlib.collections.PathCollection at 0x19d11b80>

In [320]:
plt.hist(results.resid)
Out[320]:
(array([1., 0., 2., 1., 3., 4., 1., 2., 2., 2.]),
 array([-46.44102675, -38.10480112, -29.7685755 , -21.43234987,
        -13.09612425,  -4.75989863,   3.576327  ,  11.91255262,
         20.24877825,  28.58500387,  36.92122949]),
 <a list of 10 Patch objects>)

North Location

In [343]:
#СДЕЛАЕМ ДАТАФРЕЙМ С ОТФИЛЬТРОВАННЫМИ ДАННЫМИ North Location
df_north = df.loc[(df.location == 'North')]
df_north.head()
Out[343]:
Unnamed: 0	location	town	mortality	hardness
1	2	North	Birkenhead	1668	17
3	4	North	Blackburn	1800	14
4	5	North	Blackpool	1609	18
5	6	North	Bolton	1558	10
6	7	North	Bootle	1807	15
In [344]:
df_north.plot(kind='scatter', x='mortality', y='hardness')
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
Out[344]:
<matplotlib.axes._subplots.AxesSubplot at 0x1a011880>

In [296]:
sns.pairplot(df_north) # используем pairplot для изучения взаимосвязи признаков в North Location
# Видим, что графики чуть отличаются от аналогичных графиков South Location, в более линейную сторону 
# по нижней границу hardness до 20,
# но по итогу взаимосвязь между жесткостью воды и смертностью - тоже явно не прослеживается
Out[296]:
<seaborn.axisgrid.PairGrid at 0x195bc790>

In [297]:
# Рассчитаем коэффициент корреляции Пирсона в North Location
df_north[['mortality','hardness']].corr
Out[297]:
<bound method DataFrame.corr of     mortality  hardness
1        1668        17
3        1800        14
4        1609        18
5        1558        10
6        1807        15
8        1637        10
11       1755        12
15       1491        20
16       1555        39
17       1428        39
20       1723        44
21       1379        94
22       1742         8
23       1574         9
24       1569        91
26       1591        16
28       1772        15
29       1828         8
30       1704        26
31       1702        44
35       1427        27
36       1724         6
40       1696         6
42       1711        13
43       1444        14
44       1591        49
45       1987         8
46       1495        14
49       1587        75
50       1713        71
51       1557        13
52       1640        57
53       1709        71
55       1625        20
60       1378        71>
In [298]:
# Рассчитаем коэффициент корреляции Спирмена в North Location
df_north[['mortality','hardness']].corr(method = 'spearman')
# видим разницу в меньшую сторону по абсолютному значению величины  по сравнению с общим датафреймом df,
#где присуствовали данными по обоим регионам South и North Location (|-0,40|<|-0,44|)
# и в еще меньшую сторону относительно аналогичного расчета по South Location (|-0,40|<|-0,45|)
Out[298]:
mortality	hardness
mortality	1.000000	-0.404208
hardness	-0.404208	1.000000
In [299]:
# Рассчитаем коэффициент корреляции Кенделла в North Location
df_north[['mortality','hardness']].corr(method = 'kendall')
# видим разницу в меньшую сторону по абсолютному значению величины по сравнению с общим датафреймом df,
#где присуствовали данными по обоим регионам South и North Location (|-0,28|<|-0,60|)
# и в еще меньшую сторону относительно аналогичного расчета по South Location (|-0,28|<|-0,60|)
Out[299]:
mortality	hardness
mortality	1.000000	-0.283058
hardness	-0.283058	1.000000
In [300]:
# строим тепловую карту North Location
sns.set(rc={'figure.figsize':(5.0,4.0)})
sns.heatmap(df_north.corr(),annot= True)
Out[300]:
<matplotlib.axes._subplots.AxesSubplot at 0x199863a0>

In [345]:
# Построим модель линейной регрессии для North Location
X2 = df_north[['mortality']]
y2 = df_north['hardness']
In [346]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.30, random_state=42)
In [347]:
X2_train.shape
Out[347]:
(24, 1)
In [348]:
y2_train.shape
Out[348]:
(24,)
In [349]:
model2 = LinearRegression()
model2.fit(X2_train, y2_train)  # метод обучается на данных и подбирает оптимальные коэффициенты
Out[349]:
LinearRegression()
In [350]:
model2.coef_
Out[350]:
array([-0.09774218])
In [351]:
model2.intercept_
Out[351]:
191.72797679112006
In [352]:
y2_pred = model2.predict(X2_test)
y2_pred
Out[352]:
array([-2.48573411, 37.88178607, 50.58826942, 23.22045913, 36.22016902,
       24.29562311, 25.37078708, 21.4610999 , 39.73888749, 18.52883451,
       52.1521443 ])
In [353]:
model2.score(X2_test, y2_test) # метод возвращает значение коэффициента детерминации
Out[353]:
-0.34863672627416675
In [354]:
plt.scatter(X2_test, y2_test)
plt.plot(X2_test, y2_pred, c='r')
# видим нелинейную регрессию.
# связь между параметрами также отсутствует
Out[354]:
[<matplotlib.lines.Line2D at 0x1a080b50>]

In [355]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
In [356]:
X2_const = sm.add_constant(X2_train) # техническая особенность библиотек, надо руками добавить константу
In [357]:
X2_const.shape
Out[357]:
(24, 2)
In [358]:
model2 = sm.OLS(y2_train, X2_const)
results = model2.fit()
print(results.summary())
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               hardness   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     5.269
Date:                Sat, 21 Nov 2020   Prob (F-statistic):             0.0316
Time:                        23:45:25   Log-Likelihood:                -111.11
No. Observations:                  24   AIC:                             226.2
Df Residuals:                      22   BIC:                             228.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        191.7280     69.297      2.767      0.011      48.015     335.441
mortality     -0.0977      0.043     -2.295      0.032      -0.186      -0.009
==============================================================================
Omnibus:                        2.755   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.252   Jarque-Bera (JB):                2.318
Skew:                           0.669   Prob(JB):                        0.314
Kurtosis:                       2.274   Cond. No.                     2.13e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
In [359]:
# посмотрим на остатки
plt.scatter(X2_const.iloc[:, 1], results.resid)
Out[359]:
<matplotlib.collections.PathCollection at 0x1a05a880>

In [360]:
plt.hist(results.resid)
Out[360]:
(array([5., 3., 4., 4., 0., 2., 2., 0., 2., 2.]),
 array([-31.60341826, -23.18012614, -14.75683401,  -6.33354188,
          2.08975025,  10.51304238,  18.93633451,  27.35962664,
         35.78291877,  44.2062109 ,  52.62950303]),
 <a list of 10 Patch objects>)

In [ ]:
#ОТВЕТ: линейной зависимости жёсткостью воды и средней годовой смертностью - нет, 
#ни по отношению к совокупности South и North Location, ни для каждого их них в отдельности